In [1]:
#importing the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime as dt
from impyute.imputation.cs import mice
from missingpy import MissForest
from fancyimpute import IterativeImputer
from mlens.ensemble import SuperLearner
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
[MLENS] backend: threading


In [ ]:
df = pd.read_csv("features_on_train.csv")

In [ ]:
df.head()

In [ ]:
list=[]
for i in range(len(df)):
    list.append(dt.strptime(df['last_rating_date'][i],"%Y-%m-%d"))
df['last_rating_date']=np.array(list)

list=[]
for i in range(len(df)):
    list.append(dt.strptime(df['first_rating_date'][i],"%Y-%m-%d"))
df['first_rating_date']=np.array(list)

In [ ]:
list=[]
for i in range(len(df)):
    s=str(df['first_remark_date'][i])
    if s != 'nan': list.append(dt.strptime(s,"%Y-%m-%d"))
    else: list.append(np.nan)
df['first_remark_date']=np.array(list)

list=[]
for i in range(len(df)):
    s=str(df['last_remark_date'][i])
    if s != 'nan': list.append(dt.strptime(s,"%Y-%m-%d"))
    else: list.append(np.nan)
df['last_remark_date']=np.array(list)

In [ ]:
list=[]
for i in range(len(df)):
    d = df['last_rating_date'][i]-df['first_rating_date'][i]
    list.append(d.days)
df['diff1']=np.array(list)

list=[]
for i in range(len(df)):
    d1 = df['last_remark_date'][i]
    d2 = df['first_remark_date'][i]
    if d1==np.nan or d2==np.nan : list.append(np.nan)
    else :
        d=d1-d2
    list.append(d.days)
df['diff2']=np.array(list)

In [ ]:
Y = df['left']
X = df.drop(columns=['last_rating_date','first_rating_date','last_remark_date','first_remark_date','left'])

In [ ]:
X_ID = X['my_id']
X = X.drop(columns=['my_id'])

In [ ]:
imputer=MissForest()  #imputing using miss forest and separating the test and train data set
df1 = imputer.fit_transform(X)
X = pd.DataFrame(df1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
# feature Scaling
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
X_train=sc_x.fit_transform(X_train)
X_test=sc_x.transform(X_test)

In [ ]:
#Over_sampling

smt=SMOTE()
X_train,Y_train=smt.fit_sample(X_train,Y_train)
# XGB classifier
from xgboost import XGBClassifier
classifier=XGBClassifier(n_estimators=200)

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier1=LogisticRegression(random_state=0)

In [ ]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
classifier2=RandomForestClassifier()
#SVM
from sklearn.svm import SVC
#AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
adaboost=AdaBoostClassifier()
#NaiveBayes 
from sklearn.naive_bayes import GaussianNB
naivebayes=GaussianNB()
#ensemble

In [ ]:
#Voting
from sklearn.ensemble import VotingClassifier
model=VotingClassifier(estimators=[('lr',classifier1),('xgb',classifier),('xgb1',classifier),('xgb2',classifier),('rf',classifier2),('nb1',naivebayes),('rf1',classifier2),('ad1',adaboost),
                                   ('xgb3',classifier),('xgb4',classifier)],voting='hard')
model.fit(X_train,Y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
y_test_pred=model.predict(X_test)
cm=confusion_matrix(Y_test,y_test_pred)
print(cm)
a=cm[0][0]+cm[1][1]
b=cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1]
print(a/b)

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score


ans=balanced_accuracy_score(Y_test,y_test_pred)
print(ans)

f1=f1_score(Y_test,y_test_pred)
print(f1,'f1')